# Kaggle F1 Data Collector (1950-2024)

Load raw CSV files from Kaggle (results, drivers, races, and constructors), then print the full dataset of F1 results from 1950 to 2024. After that, sort the results chronologically by year and races, then filter relevant (modern) data from 2014 to 2023.

In [50]:
import shutil

# delete old f1_cache folder
shutil.rmtree('/content/f1_cache', ignore_errors=True)
print("✅ Deleted f1_cache folder completely.")

✅ Deleted f1_cache folder completely.


In [51]:
import os, fastf1

# re-initialise new fastf1 cache
os.makedirs('/content/f1_cache', exist_ok=True)
fastf1.Cache.enable_cache('/content/f1_cache')
print("🆕 New FastF1 cache initialized.")

🆕 New FastF1 cache initialized.


In [52]:
import pandas as pd

# 🏎️ STEP 1: LOAD RAW CSV FILES
print("📂 Loading Kaggle F1 dataset files...")

kaggle_results = pd.read_csv('kaggle/results.csv')
races = pd.read_csv('kaggle/races.csv')[['raceId', 'year', 'name']]
drivers = pd.read_csv('kaggle/drivers.csv')[['driverId', 'driverRef', 'surname']]
constructors = pd.read_csv('kaggle/constructors.csv')[['constructorId', 'name']]

print("✅ All source files loaded successfully.")

# 🧩 STEP 2: MERGE INTO ONE COMPLETE DATAFRAME
merged = (
    kaggle_results
    .merge(races, on='raceId')
    .merge(drivers, on='driverId')
    .merge(constructors, on='constructorId')
)

# rename columns for clarity
merged = merged.rename(columns={
    'year': 'Year',
    'name_x': 'Race',
    'surname': 'Driver',
    'name_y': 'Constructor',
    'positionOrder': 'Position',
    'points': 'Points'
})

# save merged dataset
merged.to_csv('/content/formula1_results_1950_2024.csv', index=False)
print("💾 Merged dataset saved as formula1_results_1950_2024.csv")

# 🧹 STEP 3: CLEAN AND SORT DATA
print("🧹 Cleaning and sorting dataset...")

df = pd.read_csv('/content/formula1_results_1950_2024.csv')

# select and reorder relevant columns
df = df[['Year', 'Race', 'Driver', 'Constructor', 'Position', 'Points']]

# drop missing / invalid rows
df = df.dropna(subset=['Year', 'Race', 'Driver', 'Constructor', 'Points'])
df = df[df['Position'] != '\\N']

# ensure numeric columns have proper data types
df['Year'] = df['Year'].astype(int)
df['Position'] = df['Position'].astype(int)
df['Points'] = df['Points'].astype(float)

# sort chronologically
df = df.sort_values(by=['Year', 'Race']).reset_index(drop=True)

print(f"✅ Cleaned dataset shape: {df.shape}")

# save the clean version
df.to_csv('/content/formula1_results_cleaned.csv', index=False)
print("💾 Saved clean dataset as formula1_results_cleaned.csv")

# 🎯 STEP 4: FILTER MODERN ERA (2014–2023)
# skip 2024 due to partial/incomplete results
df_filtered = df[df['Year'].between(2014, 2023)]
df_filtered.to_csv('/content/f1_results_2014_2023.csv', index=False)
print(f"🏁 Saved modern-era filtered dataset (2014–2023) → {len(df_filtered)} rows")


📂 Loading Kaggle F1 dataset files...
✅ All source files loaded successfully.
💾 Merged dataset saved as formula1_results_1950_2024.csv
🧹 Cleaning and sorting dataset...
✅ Cleaned dataset shape: (26759, 6)
💾 Saved clean dataset as formula1_results_cleaned.csv
🏁 Saved modern-era filtered dataset (2014–2023) → 4147 rows
